### Заметки:

#### Нам

- Попросить у лектора примеры значений всех выдаваемых величин и результатов (хотя бы по порядку величины)

- N_d, N_d0 ??? (относительно семинара)

- В чем измеряется диэлектрическая проницаемость?

- 

#### В fompy

- Почему нет спиновой двойки в выражении для Nd+ (В семинаре есть) 
- Откуда отсчитывается E донора?

In [1]:
import math
from fompy import constants
from fompy import functions
from fompy import materials
from fompy import models
from fompy import units
from fompy import util

Входные данные

In [12]:
#условия
T = 300 #Температура, K
E_out = 0 #Внешнее поле, B/м

#полупроводник
E_gap = 1.12 * constants.eV #Между валентной зоной и зоной проводимости, эрг
epsilon = 11.7 #диэлекрическая проницаемость
m_h = 0.81 * constants.me #эффективаная масса дырок, г
m_e = 0.36 * constants.me #эффективная масса электронов, г

#легирование (доноры)
E_d = 0 #положение уровня донора (от дна зоны проводимости!)
N_d0 = 0 #концентрация доноров

#поверхностные акцепторы
N_as = 0 #плотность поверхностных акцепторов
E_as = 0 #их положение уровня энергии

Создаю наш полупроводник

In [14]:
pure_scond = models.Semiconductor(m_e, m_h, E_gap, chi=None, eps=epsilon)
doped = models.DopedSemiconductor(mat=pure_scond, Na=0, Ea=0, Nd=N_d0, Ed=E_d)


1. Сначала находим эффективную плотность состояний для электронов и дырок


Формула лектора: 

$$N_{C(V)} = 2.51\cdot 10^{18} (\frac{m_{C(V)}}{m_0})^{3/2} (\frac{T}{300})^{3/2} \cdot cm^{-3}$$

Формула fompy:

$$N_{C(V)} = 2 \left( \frac{2 \pi m_{C(V)} k T }{ (2 \pi \hbar)^2 } \right)^{3/2} \cdot cm^{-3}$$


*Сходится по обеим формулам*

In [21]:
N_c = doped.Nc(T) #cm^-3
N_v = doped.Nv(T) #cm^-3
N_c, N_v

(5.4154558360038e+18, 1.8277163446512824e+19)

2. Находим положение уровня Ферми в квазинейтральном объеме из условия электронейтральности. Формулы лектора:
$$N_d^+ + p = n$$

$$n = N_c \cdot e^{\frac{E_f - E_g}{kT}}$$
$$p = N_v \cdot e^{\frac{-E_f}{kT}}$$

$$N_d^+ = N_{d0} \frac{1}{1 + e^{\frac{E_g - E_d - E_f}{kT}}}$$

Формулы fompy:

$$N_d^+ = N_d \cdot (1 - f(E_d))$$

$$n = N_c(T) \Phi_{1/2}\left( \frac{ E_f - E_g }{ k T } \right)$$

$$n_p = N_v(T) \Phi_{1/2}\left( \frac{ - E_f }{ k T } \right)$$

$$p + N_d^{+} - n - N_a^{-} = 0$$

In [25]:
E_f = doped.fermi_level(T)
E_f/constants.eV

0.5757279968261719

Пока не ввели параметры примеси, совпадает со значением для чистого:

In [27]:
pure_scond.fermi_level(T)/constants.eV

0.5757279968261719